# Importing bank transactions in Python using Pandas

If you want to analyze your expenses in Python, the first step is importing your financial transactions.

From your bank account you should be able to download a file containing all transactions in a period. The format of the file is often one of the following:

* an Excel file
* a CSV file
* a PDF file
* a TXT file
* a QIF file

What's the best extension? If you want to import transactions flawlessly in a DataFrame, you should download a CSV file; if this extension is not available, download the Excel one; if this extension is not available, download the QIF file; if this extension is not available, download the TXT file. Do not download the PDF file.

Filter your transaction using the largest period available and download the file.

## If you download a CSV file

The `read_csv` function allows you to read a CSV file into DataFrame.

Let's see some examples.

In [12]:
import pandas as pd

transactions = pd.read_csv(filepath_or_buffer="transactions-files/01.csv")

print(transactions)

         date                        description   amount
0  2024-04-04  My Online Shop XYZ - Invoice 1234  -123.45
1  2024-04-03  My Online Shop ABC - Invoice 5678   -54.32
2  2024-04-02           My Employer - April 2024  1234.56


Here we have a rather correctly formatted CSV file and we have only to pass the argument `filepath_or_buffer` to the `read_csv` function to indicate the path of the CSV file to read. Our DataFrame is ready to be analyzed.

What if the file need some cleaning? Let's see another example.

In [13]:
transactions = pd.read_csv(filepath_or_buffer="transactions-files/02.csv")

print(transactions)

   transaction date transaction description detailed description amount ($)
0          20240404      My Online Shop XYZ         Invoice 1234    -123.45
1          20240403      My Online Shop ABC         Invoice 5678     -54.32
2          20240402             My Employer           April 2024   1,234.56


Here we can see different "problems":
* dates are awkwardly formatted
* descriptions are spread in two columns, "transaction description" and "detailed description"
* amounts have a thousands separator (",")

We need to "clean" this data. How to do that?

### Parse non-standard dates

To parse dates in this format, we need to pass two arguments to the `read_csv` function, `date_format` (the format to use for parsing dates) and `parse_dates` (the list of columns to parse).

In [19]:
transactions = pd.read_csv(filepath_or_buffer="transactions-files/02.csv",parse_dates=[0],date_format="%Y%m%d")

print(transactions)

  transaction date transaction description detailed description amount ($)
0       2024-04-04      My Online Shop XYZ         Invoice 1234    -123.45
1       2024-04-03      My Online Shop ABC         Invoice 5678     -54.32
2       2024-04-02             My Employer           April 2024   1,234.56


The dates are parsed correctly.

### Concatenate multiple description columns

To concatenate multiple columns we can replace the column "transaction description" with the concatenation of the columns "transaction description" and "detailed description" and remove the column  "detailed description".

In [26]:
transactions = pd.read_csv(filepath_or_buffer="transactions-files/02.csv",parse_dates=[0],date_format="%Y%m%d")

# concatenate descriptions columns
transactions["transaction description"]=transactions["transaction description"]+" - "+transactions["detailed description"]
transactions.drop("detailed description", axis=1, inplace=True)

print(transactions)

  transaction date            transaction description amount ($)
0       2024-04-04  My Online Shop XYZ - Invoice 1234    -123.45
1       2024-04-03  My Online Shop ABC - Invoice 5678     -54.32
2       2024-04-02           My Employer - April 2024   1,234.56


We now have a single description column.

### Parse amounts with the thousand separator

To parse amounts correctly we have to use another argument of the `read_csv` function, `thousands`.

In [27]:
transactions = pd.read_csv(filepath_or_buffer="transactions-files/02.csv",parse_dates=[0],date_format="%Y%m%d",thousands=",")

# concatenate descriptions columns
transactions["transaction description"]=transactions["transaction description"]+" - "+transactions["detailed description"]
transactions.drop("detailed description", axis=1, inplace=True)

print(transactions)

  transaction date            transaction description  amount ($)
0       2024-04-04  My Online Shop XYZ - Invoice 1234     -123.45
1       2024-04-03  My Online Shop ABC - Invoice 5678      -54.32
2       2024-04-02           My Employer - April 2024     1234.56


Amounts are now parsed correctly.

What if we want to rename headers to simplify DataFrame manipulations?

In [28]:
transactions = pd.read_csv(filepath_or_buffer="transactions-files/02.csv",parse_dates=[0],date_format="%Y%m%d",thousands=",")

# rename columns
transactions.rename(columns={"transaction date": "date", "transaction description": "description", "detailed description":"detailed", "amount ($)": "amount"}, inplace=True)

# concatenate descriptions columns
transactions["description"]=transactions["description"]+" - "+transactions["detailed"]
transactions.drop("detailed", axis=1, inplace=True)

print(transactions)

        date                        description   amount
0 2024-04-04  My Online Shop XYZ - Invoice 1234  -123.45
1 2024-04-03  My Online Shop ABC - Invoice 5678   -54.32
2 2024-04-02           My Employer - April 2024  1234.56
